In [1]:
from __future__ import print_function, division
from collections import Counter
import csv
import numpy as np
import random
import sys
import os
import copy
import time
from datetime import datetime,timedelta
from math import log, sqrt
import pandas as pd
import distance
from keras.models import load_model
import matplotlib.pyplot as plt
from jellyfish._jellyfish import damerau_levenshtein_distance

# 1.Read file

In [134]:
eventlog = "env_permit.csv"
eventlog_name = "env_permit"

In [135]:
csvfile = open('../data/%s' % eventlog, 'r')
spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
next(spamreader, None)  # skip the headers
ascii_offset = 161

# 2.Preprocessing

In [136]:
lines = [] #these are all the activity seq
timeseqs = [] #time sequences (differences between two events)
timeseqs2 = [] #time sequences (differences between the current and first)

#helper variables
lastcase = ''
line = ''
firstLine = True
times = []
times2 = []
numlines = 0
casestarttime = None
lasteventtime = None

In [137]:
for row in spamreader: #the rows are "CaseID,ActivityID,CompleteTimestamp"
    t = datetime.strptime(row[2], "%Y-%m-%d %H:%M:%S") #creates a datetime object from row[2]
    if row[0]!=lastcase:  #'lastcase' is to save the last executed case for the loop
        casestarttime = t
        lasteventtime = t
        lastcase = row[0]
        if not firstLine:
            lines.append(line)
            timeseqs.append(times)
            timeseqs2.append(times2)
        line = ''
        times = []
        times2 = []
        numlines+=1
    line+=chr(int(row[1])+ascii_offset)
    timesincelastevent = t - lasteventtime
    timesincecasestart = t - casestarttime
    timediff = 86400 * timesincelastevent.days + timesincelastevent.seconds  #time b/t current and last event
    timediff2 = 86400 * timesincecasestart.days + timesincecasestart.seconds #time b/t current and starting event
    times.append(timediff)
    times2.append(timediff2)
    lasteventtime = t
    firstLine = False

# add last case
lines.append(line)
timeseqs.append(times)
timeseqs2.append(times2)
numlines+=1

In [138]:
# Example of accessing processed data
#print(lines)
#print(timeseqs)
#print(timeseqs2)

In [139]:
#average time between events
divisor = np.mean([item for sublist in timeseqs for item in sublist]) 
print('divisor: {}'.format(divisor))
#average time between current and starting events
divisor2 = np.mean([item for sublist in timeseqs2 for item in sublist]) 
print('divisor2: {}'.format(divisor2))


divisor: 186908.12533381264
divisor2: 5159390.008756163


In [140]:
# separate training data into 3 parts
elems_per_fold = int(round(numlines/3)) #calculate the number of elements per fold
# fist 1/3 elements and their calculated time features
fold1 = lines[:elems_per_fold]
fold1_t = timeseqs[:elems_per_fold]
fold1_t2 = timeseqs2[:elems_per_fold]
# second 1/3 elements and their calculated time features
fold2 = lines[elems_per_fold:2*elems_per_fold]
fold2_t = timeseqs[elems_per_fold:2*elems_per_fold]
fold2_t2 = timeseqs2[elems_per_fold:2*elems_per_fold]
# last 1/3 elements and their calculated time features
fold3 = lines[2*elems_per_fold:]
fold3_t = timeseqs[2*elems_per_fold:]
fold3_t2 = timeseqs2[2*elems_per_fold:]

#consider only fist and second part as training set, leave away fold3 for now
lines = fold1 + fold2
lines_t = fold1_t + fold2_t
lines_t2 = fold1_t2 + fold2_t2

In [141]:
step = 1
sentences = []
softness = 0
next_chars = []
lines = list(map(lambda x: x+ '!',lines)) #add a delimiter symbol '!' to the end of each line
maxlen = max(map(lambda x: len(x),lines)) #find maximum line size

# next lines here to get all possible characters for events and annotate them with numbers
chars = list(map(lambda x: set(x),lines))
chars = list(set().union(*chars))
chars.sort()

In [142]:
target_chars = copy.copy(chars)

if '!' in chars:
    chars.remove('!')
print('total chars: {}, target chars: {}'.format(len(chars), len(target_chars)))

total chars: 299, target chars: 300


In [143]:
#get the target chars from the training set
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
target_char_indices = dict((c, i) for i, c in enumerate(target_chars))
target_indices_char = dict((i, c) for i, c in enumerate(target_chars))
print(indices_char)

{0: '¢', 1: '£', 2: '¤', 3: '¥', 4: '¦', 5: '§', 6: '¨', 7: '©', 8: 'ª', 9: '«', 10: '¬', 11: '\xad', 12: '®', 13: '¯', 14: '°', 15: '±', 16: '²', 17: '³', 18: '´', 19: 'µ', 20: '¶', 21: '·', 22: '¸', 23: '¹', 24: 'º', 25: '»', 26: '¼', 27: '½', 28: '¾', 29: '¿', 30: 'À', 31: 'Á', 32: 'Â', 33: 'Ä', 34: 'Å', 35: 'Æ', 36: 'Ç', 37: 'È', 38: 'É', 39: 'Ê', 40: 'Ì', 41: 'Î', 42: 'Ï', 43: 'Ð', 44: 'Ñ', 45: 'Ò', 46: 'Ó', 47: 'Ô', 48: 'Õ', 49: 'Ö', 50: '×', 51: 'Ø', 52: 'Ù', 53: 'Ú', 54: 'Û', 55: 'Ü', 56: 'Ý', 57: 'Þ', 58: 'ß', 59: 'à', 60: 'á', 61: 'ã', 62: 'ä', 63: 'å', 64: 'æ', 65: 'ç', 66: 'è', 67: 'é', 68: 'ê', 69: 'ë', 70: 'ì', 71: 'í', 72: 'î', 73: 'ï', 74: 'ð', 75: 'ò', 76: 'ó', 77: 'ô', 78: 'õ', 79: 'ö', 80: '÷', 81: 'ø', 82: 'ú', 83: 'û', 84: 'ü', 85: 'ý', 86: 'ÿ', 87: 'Ā', 88: 'ā', 89: 'Ă', 90: 'ą', 91: 'Ć', 92: 'ć', 93: 'Ĉ', 94: 'ĉ', 95: 'č', 96: 'Ď', 97: 'ď', 98: 'Đ', 99: 'Ē', 100: 'ē', 101: 'ĕ', 102: 'Ė', 103: 'Ę', 104: 'Ě', 105: 'ě', 106: 'Ĝ', 107: 'ĝ', 108: 'Ğ', 109: 'Ġ', 110: '

# 3. Feature Enginnering

In [144]:
csvfile = open('../data/%s' % eventlog, 'r')
spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
next(spamreader, None)  # skip the headers
lastcase = ''
line = ''
firstLine = True
lines = []
timeseqs = []
timeseqs2 = []
timeseqs3 = []
timeseqs4 = []
times = []
times2 = []
times3 = []
times4 = []
numlines = 0
casestarttime = None
lasteventtime = None

In [145]:
for row in spamreader:
    t = time.strptime(row[2], "%Y-%m-%d %H:%M:%S")
    if row[0]!=lastcase:
        casestarttime = t
        lasteventtime = t
        lastcase = row[0]
        if not firstLine:
            lines.append(line)
            timeseqs.append(times)
            timeseqs2.append(times2)
            timeseqs3.append(times3)
            timeseqs4.append(times4)
        line = ''
        times = []
        times2 = []
        times3 = []
        times4 = []
        numlines+=1
    line+=chr(int(row[1])+ascii_offset)
    timesincelastevent = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(lasteventtime))
    timesincecasestart = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(casestarttime))
    midnight = datetime.fromtimestamp(time.mktime(t)).replace(hour=0, minute=0, second=0, microsecond=0)
    timesincemidnight = datetime.fromtimestamp(time.mktime(t))-midnight
    timediff = 86400 * timesincelastevent.days + timesincelastevent.seconds
    timediff2 = 86400 * timesincecasestart.days + timesincecasestart.seconds
    timediff3 = timesincemidnight.seconds #this leaves only time even occurred after midnight
    timediff4 = datetime.fromtimestamp(time.mktime(t)).weekday() #day of the week
    times.append(timediff)
    times2.append(timediff2)
    times3.append(timediff3)
    times4.append(timediff4)
    lasteventtime = t
    firstLine = False

# add last case
lines.append(line)
timeseqs.append(times)
timeseqs2.append(times2)
timeseqs3.append(times3)
timeseqs4.append(times4)
numlines+=1

In [149]:
# fold 1
elems_per_fold = int(round(numlines/3)) #calculate the number of elements per fold
fold1 = lines[:elems_per_fold]
fold1_t = timeseqs[:elems_per_fold]
fold1_t2 = timeseqs2[:elems_per_fold]
fold1_t3 = timeseqs3[:elems_per_fold]
fold1_t4 = timeseqs4[:elems_per_fold]
with open(f'output_files/folds/{eventlog_name}_fold1.csv', 'w', newline='', encoding='utf-8') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row, timeseq in zip(fold1, fold1_t):
        spamwriter.writerow([str(s) + '#{}'.format(t).encode('unicode_escape').decode('utf-8') for s, t in zip(row, timeseq)])
        
# fold 2
fold2 = lines[elems_per_fold:2*elems_per_fold]
fold2_t = timeseqs[elems_per_fold:2*elems_per_fold]
fold2_t2 = timeseqs2[elems_per_fold:2*elems_per_fold]
fold2_t3 = timeseqs3[elems_per_fold:2*elems_per_fold]
fold2_t4 = timeseqs4[elems_per_fold:2*elems_per_fold]
with open(f'output_files/folds/{eventlog_name}_fold2.csv', 'w', newline='', encoding='utf-8') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row, timeseq in zip(fold2, fold2_t):
        spamwriter.writerow([str(s) + '#{}'.format(t).encode('unicode_escape').decode('utf-8') for s, t in zip(row, timeseq)])
        
# fold 3
fold3 = lines[2*elems_per_fold:]
fold3_t = timeseqs[2*elems_per_fold:]
fold3_t2 = timeseqs2[2*elems_per_fold:]
fold3_t3 = timeseqs3[2*elems_per_fold:]
fold3_t4 = timeseqs4[2*elems_per_fold:]
with open(f'output_files/folds/{eventlog_name}_fold3.csv', 'w', newline='', encoding='utf-8') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row, timeseq in zip(fold3, fold3_t):
        spamwriter.writerow([str(s) + '#{}'.format(t).encode('unicode_escape').decode('utf-8') for s, t in zip(row, timeseq)])

lines = fold1 + fold2
lines_t = fold1_t + fold2_t
lines_t2 = fold1_t2 + fold2_t2
lines_t3 = fold1_t3 + fold2_t3
lines_t4 = fold1_t4 + fold2_t4

In [150]:
step = 1
sentences = []
softness = 0
next_chars = []
lines = list(map(lambda x: x+'!', lines))

sentences_t = []
sentences_t2 = []
sentences_t3 = []
sentences_t4 = []
next_chars_t = []
next_chars_t2 = []
next_chars_t3 = []
next_chars_t4 = []

for line, line_t, line_t2, line_t3, line_t4 in zip(lines, lines_t, lines_t2, lines_t3, lines_t4):
    for i in range(0, len(line), step):
        if i==0:
            continue

        #we add iteratively, first symbol of the line, then two first, three...
        sentences.append(line[0: i])
        sentences_t.append(line_t[0:i])
        sentences_t2.append(line_t2[0:i])
        sentences_t3.append(line_t3[0:i])
        sentences_t4.append(line_t4[0:i])
        next_chars.append(line[i])

        if i==len(line)-1: # special case to deal time of end character
            next_chars_t.append(0)
            next_chars_t2.append(0)
            next_chars_t3.append(0)
            next_chars_t4.append(0)
        else:
            next_chars_t.append(line_t[i])
            next_chars_t2.append(line_t2[i])
            next_chars_t3.append(line_t3[i])
            next_chars_t4.append(line_t4[i])

print('nb sequences:', len(sentences))

nb sequences: 24600


# 4. Encoding

In [151]:
print('Vectorization...')
num_features = len(chars)+5
print('num features: {}'.format(num_features))
X = np.zeros((len(sentences), maxlen, num_features), dtype=np.float32)
y_a = np.zeros((len(sentences), len(target_chars)), dtype=np.float32)
y_t = np.zeros((len(sentences)), dtype=np.float32)
for i, sentence in enumerate(sentences):
    leftpad = maxlen-len(sentence)
    next_t = next_chars_t[i]
    sentence_t = sentences_t[i]
    sentence_t2 = sentences_t2[i]
    sentence_t3 = sentences_t3[i]
    sentence_t4 = sentences_t4[i]
    for t, char in enumerate(sentence):
        multiset_abstraction = Counter(sentence[:t+1])
        for c in chars:
            if c==char: #this will encode present events to the right places
                X[i, t+leftpad, char_indices[c]] = 1
        X[i, t+leftpad, len(chars)] = t+1
        X[i, t+leftpad, len(chars)+1] = sentence_t[t]/divisor
        X[i, t+leftpad, len(chars)+2] = sentence_t2[t]/divisor2
        X[i, t+leftpad, len(chars)+3] = sentence_t3[t]/86400
        X[i, t+leftpad, len(chars)+4] = sentence_t4[t]/7
    for c in target_chars:
        if c==next_chars[i]:
            y_a[i, target_char_indices[c]] = 1-softness
        else:
            y_a[i, target_char_indices[c]] = softness/(len(target_chars)-1)
    y_t[i] = next_t/divisor
    np.set_printoptions(threshold=sys.maxsize)

Vectorization...
num features: 304


In [152]:
# reshape the training set to 2D
X = X.reshape(X.shape[0], -1)

# 5. Train Random Forest

In [153]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score

In [154]:
# Define the parameter grid for grid search
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [None, 5, 10],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]  # Minimum number of samples required to be at a leaf node
}

In [155]:
# Create the random forest classifier
rf_model = RandomForestClassifier(n_estimators=100,random_state=42)

In [ ]:
# Perform grid search to find the best hyperparameters
random_search = RandomizedSearchCV(rf_model, param_grid, cv=5, scoring='accuracy', verbose=2)
random_search.fit(X, y_a)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END max_depth=5, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time= 3.8min
[CV] END max_depth=5, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time= 3.8min
[CV] END max_depth=5, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time= 3.9min
[CV] END max_depth=5, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time= 3.8min
[CV] END max_depth=5, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time= 3.9min
[CV] END max_depth=None, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time= 8.4min
[CV] END max_depth=None, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time= 8.8min
[CV] END max_depth=None, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time= 8.2min
[CV] END max_depth=None, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time= 7.9min
[CV] END max_depth=None, mi

In [ ]:
# Get the best hyperparameters and model
best_params = random_search.best_params_
best_model = random_search .best_estimator_
print(best_params,best_model)

In [ ]:
best_model.fit(X, y_a)

In [ ]:
import joblib

# Save the model with the best hyperparameters
joblib.dump(best_model, f'output_files/models/{eventlog_name}_RF_best_model.pkl')